In [0]:
%sql
create or replace temp view icdcodes using parquet
options(path "/mnt/bronze/icd_codes");

select * from icdcodes;

In [0]:

%sql
CREATE TABLE IF NOT EXISTS `hrcm-data-catalog`.silver.icd_codes (
    icd_code STRING,
    icd_code_type STRING,
    code_description STRING,
    inserted_date DATE,
    updated_date DATE,
    is_current_flag BOOLEAN
)
   

In [0]:

%sql
MERGE INTO
  `hrcm-data-catalog`.silver.icd_codes AS target
USING
  icdcodes AS source
ON target.icd_code = source.icd_code
WHEN MATCHED AND
  target.code_description != source.code_description
  THEN UPDATE SET
  target.code_description = source.code_description,
  target.updated_date = source.updated_date,
  target.is_current_flag = False
WHEN NOT MATCHED THEN INSERT (
    icd_code, icd_code_type, code_description, inserted_date, updated_date, is_current_flag
  )
  VALUES (
    source.icd_code,
    source.icd_code_type,
    source.code_description,
    source.inserted_date,
    source.updated_date,
    source.is_current_flag
  )

In [0]:
df_silver = spark.sql("SELECT * FROM `hrcm-data-catalog`.silver.icd_codes")
df_silver.write.format("delta").mode("overwrite").save("/mnt/silver/icd_codes")